<a href="https://colab.research.google.com/github/sisomimoctrl/finance-data/blob/main/investing_realtime_us_spx_500_futures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import json
import datetime
headers = {'User-Agent':'Chrome/78.0.3904.87 Safari/537.36',}
url = 'https://api.investing.com/api/financialdata/8839/historical/chart/?interval=PT5M&pointscount=70'
json_data = json.loads(requests.get(url, headers=headers).text)
json_data

{'data': [[1655849400000, 3763.25, 3764, 3762.25, 3764, 318, 2363],
  [1655849700000, 3763.75, 3763.75, 3762.25, 3763, 283, 2681],
  [1655850000000, 3763.25, 3765.5, 3762, 3765.25, 596, 2964],
  [1655850300000, 3765.5, 3765.5, 3764, 3764.5, 571, 3560],
  [1655850600000, 3764.5, 3764.5, 3763, 3763.5, 425, 4131],
  [1655850900000, 3764, 3764.25, 3761.75, 3762.25, 422, 4556],
  [1655851200000, 3762.5, 3764.5, 3762, 3764.5, 258, 4978],
  [1655851500000, 3764, 3765, 3763.75, 3763.75, 414, 5236],
  [1655851800000, 3764.5, 3764.5, 3764, 3764, 210, 5650],
  [1655852100000, 3764, 3765.75, 3763.5, 3765.5, 174, 5860],
  [1655852400000, 3765.5, 3765.75, 3762.75, 3763.75, 652, 6034],
  [1655852700000, 3763.75, 3764, 3762.25, 3762.25, 352, 6686],
  [1655853000000, 3762.25, 3762.5, 3761.25, 3761.5, 557, 7038],
  [1655853300000, 3762, 3762, 3760.25, 3760.25, 745, 7595],
  [1655853600000, 3760, 3761, 3759.5, 3760.75, 700, 8340],
  [1655853900000, 3760.75, 3761.75, 3760.5, 3760.5, 594, 9040],
  [1655854

In [2]:
df = pd.DataFrame(json_data['data'])
df.head()

,0,1,2,3,4,5,6
0,1655849400000,3763.25,3764.00,3762.25,3764.00,318,2363
1,1655849700000,3763.75,3763.75,3762.25,3763.00,283,2681
2,1655850000000,3763.25,3765.50,3762.00,3765.25,596,2964
3,1655850300000,3765.50,3765.50,3764.00,3764.50,571,3560
4,1655850600000,3764.50,3764.50,3763.00,3763.50,425,4131


In [3]:
df = df.loc[:,0:5]
df.columns=['Timestamp','Open','High','Low','Close','Volume']
df.head()

,Timestamp,Open,High,Low,Close,Volume
0,1655849400000,3763.25,3764.00,3762.25,3764.00,318
1,1655849700000,3763.75,3763.75,3762.25,3763.00,283
2,1655850000000,3763.25,3765.50,3762.00,3765.25,596
3,1655850300000,3765.50,3765.50,3764.00,3764.50,571
4,1655850600000,3764.50,3764.50,3763.00,3763.50,425


In [4]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
df.rename(columns = {'Timestamp':'Date'},inplace=True)
df.set_index('Date', inplace=True)
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2022-06-21 22:10:00,3763.25,3764.00,3762.25,3764.00,318
2022-06-21 22:15:00,3763.75,3763.75,3762.25,3763.00,283
2022-06-21 22:20:00,3763.25,3765.50,3762.00,3765.25,596
2022-06-21 22:25:00,3765.50,3765.50,3764.00,3764.50,571
2022-06-21 22:30:00,3764.50,3764.50,3763.00,3763.50,425


In [5]:
# convert timezone from UTC to GMT+9
df.index = df.index.tz_localize('gmt').tz_convert('Asia/Seoul')
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2022-06-22 07:10:00+09:00,3763.25,3764.00,3762.25,3764.00,318
2022-06-22 07:15:00+09:00,3763.75,3763.75,3762.25,3763.00,283
2022-06-22 07:20:00+09:00,3763.25,3765.50,3762.00,3765.25,596
2022-06-22 07:25:00+09:00,3765.50,3765.50,3764.00,3764.50,571
2022-06-22 07:30:00+09:00,3764.50,3764.50,3763.00,3763.50,425


In [6]:
import plotly.graph_objects as go
from datetime import datetime

df1 = df.reset_index()
df1['Date'] = df1['Date'].apply(lambda x : datetime.strftime(x, '%Y-%m-%d %H:%M:%S')) # Datetime to str

stock_name = 'S&P 500 선물'

fig = go.Figure(data=[go.Candlestick(x=df1['Date'],
                open=df1['Open'],
                high=df1['High'],
                low=df1['Low'],
                close=df1['Close'],
                increasing_line_color = 'red', # 상승색, default는 녹색
                decreasing_line_color = 'blue' # 하락색, default는 적색
                )])

fig.layout = dict(title=stock_name, 
                  xaxis = dict(type="category", 
                categoryorder='category ascending'))

fig.update_xaxes(nticks=3)
fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()